## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
import os
 
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
 
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization, LeakyReLU
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

Using plaidml.keras.backend backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [8]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, (3,3),padding='same',input_shape=(32,32,3)))
classifier.add(LeakyReLU())
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))
#卷積組合
classifier.add(Convolution2D(64, (3,3),padding='same'))
classifier.add(LeakyReLU())
classifier.add(BatchNormalization())
#flatten
classifier.add(Flatten())
#FC
classifier.add(Dense(100))
classifier.add(LeakyReLU())
classifier.add(BatchNormalization())
classifier.add(Dropout(p=0.5))
#輸出
classifier.add(Dense(10,activation='softmax'))
#超過兩個就要選categorical_crossentrophy

classifier.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 16, 16, 64)        0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 16, 16, 64)        256       
__________

In [9]:
classifier.fit(x_train,y_train,batch_size=100,epochs=500)

Epoch 1/500
50000/50000 [==============================] - 57s 1ms/step - loss: 1.9115 - acc: 0.3627
Epoch 2/500
50000/50000 [==============================] - 53s 1ms/step - loss: 1.6367 - acc: 0.4215
Epoch 3/500
50000/50000 [==============================] - 53s 1ms/step - loss: 1.6519 - acc: 0.4169
Epoch 4/500
13500/50000 [=======>......................] - ETA: 38s - loss: 1.6007 - acc: 0.4363ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/Users/yukaizhuo/opt/anaconda3/envs/datasci/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-ea5a92f17857>", line 1, in <module>
    classifier.fit(x_train,y_train,batch_size=100,epochs=500)
  File "/Users/yukaizhuo/opt/anaconda3/envs/datasci/lib/python3.8/site-packages/keras/engine/training.py", line 1027, in fit
    return training_

TypeError: object of type 'NoneType' has no len()

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [ ]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)